# Lab | Hypothesis Testing

**Objective**

Welcome to the Hypothesis Testing Lab, where we embark on an enlightening journey through the realm of statistical decision-making! In this laboratory, we delve into various scenarios, applying the powerful tools of hypothesis testing to scrutinize and interpret data.

From testing the mean of a single sample (One Sample T-Test), to investigating differences between independent groups (Two Sample T-Test), and exploring relationships within dependent samples (Paired Sample T-Test), our exploration knows no bounds. Furthermore, we'll venture into the realm of Analysis of Variance (ANOVA), unraveling the complexities of comparing means across multiple groups.

So, grab your statistical tools, prepare your hypotheses, and let's embark on this fascinating journey of exploration and discovery in the world of hypothesis testing!

**Challenge 1**

In this challenge, we will be working with pokemon data. The data can be found here:

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv

In [11]:
#libraries
import pandas as pd
import scipy.stats as st
import numpy as np



In [12]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/pokemon.csv")
df

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


- We posit that Pokemons of type Dragon have, on average, more HP stats than Grass. Choose the propper test and, with 5% significance, comment your findings.

In [13]:
#code here
import scipy.stats as st

alpha = 0.05

dragon_pkmn = df[(df["Type 1"] == "Dragon") | (df["Type 2"] == "Dragon")]["HP"]
grass_pkmn = df[(df["Type 1"] == "Grass") | (df["Type 2"] == "Grass")]["HP"]

# H0: dragon_pkmn["HP"] <= grass_pkmn["HP"]
# H1: dragon_pkmn["HP"] > grass_pkmn["HP"]

st.ttest_ind(dragon_pkmn, grass_pkmn, alternative="greater")


TtestResult(statistic=4.4991348531252635, pvalue=7.009713739308821e-06, df=143.0)

(pvalue=7.009713739308821e-06) < 0.05. We can accept our H1 with a confidence of 95%

- We posit that Legendary Pokemons have different stats (HP, Attack, Defense, Sp.Atk, Sp.Def, Speed) when comparing with Non-Legendary. Choose the propper test and, with 5% significance, comment your findings.


In [14]:
#code here

legendary_pkmn = df[df["Legendary"] == True][["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]]
non_legendary_pkmn = df[df["Legendary"] == False][["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]]
alpha = 0.05


for x in legendary_pkmn.columns:
    p_value = st.ttest_ind(legendary_pkmn[x], non_legendary_pkmn[x], alternative="two-sided")[1]
    if p_value<0.05:
        print(f"{x} is different with a 0.95 confidence")
    else:
        print("We can´t discard H0")


# Podria haber hecho stat, p_value = st.ttest_ind(legendary_pkmn[["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]], non_legendary_pkmn[["HP", "Attack", "Defense", "Sp. Atk", "Sp. Def", "Speed"]], alternative="two-sided") y lo
# tendría directamente un result por valor es más corto pero es mejor como lo hice


HP is different with a 0.95 confidence
Attack is different with a 0.95 confidence
Defense is different with a 0.95 confidence
Sp. Atk is different with a 0.95 confidence
Sp. Def is different with a 0.95 confidence
Speed is different with a 0.95 confidence


**Challenge 2**

In this challenge, we will be working with california-housing data. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv

In [15]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/california_housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


**We posit that houses close to either a school or a hospital are more expensive.**

- School coordinates (-118, 37)
- Hospital coordinates (-122, 34)

We consider a house (neighborhood) to be close to a school or hospital if the distance is lower than 0.50.

Hint:
- Write a function to calculate euclidean distance from each house (neighborhood) to the school and to the hospital.
- Divide your dataset into houses close and far from either a hospital or school.
- Choose the propper test and, with 5% significance, comment your findings.
 

In [16]:
import numpy as np

def is_house_near(df):
    
    long = float(df["longitude"])
    lat = float(df["latitude"])

    school_coords = (-118, 37)
    hosp_coords = (-122, 34)

    school_dist = np.sqrt((long - school_coords[0])**2 + (lat - school_coords[1])**2 )
    hosp_dist = np.sqrt((long- hosp_coords[0])**2 + (lat - hosp_coords[1])**2 )

    if ((school_dist < 0.5) | (hosp_dist < 0.5)):
        return True
    else:
        return False


list_near = []
list_far = []

for x in range(df.shape[0]):
    if is_house_near(df[df.index == x]):
        list_near.append(df.iloc[x])
    else: 
        list_far.append(df.iloc[x])

df_near = pd.DataFrame(list_near)
df_far = pd.DataFrame(list_far)



C:\Users\Iván\AppData\Local\Temp\ipykernel_17012\91080643.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  long = float(df["longitude"])
C:\Users\Iván\AppData\Local\Temp\ipykernel_17012\91080643.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  lat = float(df["latitude"])


In [17]:
# H0: price(df_near) <= price(df_far)
# H1: price(df_near) > price(df_far)

st.ttest_ind(df_near["median_house_value"], df_far["median_house_value"], alternative="greater")



TtestResult(statistic=-2.2146147257665834, pvalue=0.9866001334644356, df=16998.0)

With a (p_value = 0.9866001334644356) < 0.05 we can´t refuse H0

In [18]:
# import math
# import pandas as pd

# def is_house_near(long, lat):
#     school_coords = (-118, 37)
#     hosp_coords = (-122, 34)

#     # Calcular las distancias
#     school_dist = math.sqrt((long - school_coords[0])**2 + (lat - school_coords[1])**2)
#     hosp_dist = math.sqrt((long - hosp_coords[0])**2 + (lat - hosp_coords[1])**2)

#     return (school_dist < 0.5) or (hosp_dist < 0.5)

# list_near = []
# list_far = []

# for x in range(df.shape[0]):
#     long = float(df.iloc[x]["longitude"])
#     lat = float(df.iloc[x]["latitude"])
    
#     if is_house_near(long, lat):
#         list_near.append(df.iloc[x])
#     else: 
#         list_far.append(df.iloc[x])

# # Crear DataFrames a partir de las listas
# df_near = pd.DataFrame(list_near)
# df_far = pd.DataFrame(list_far)

# # Imprimir resultados
# print("DataFrame de casas cercanas:")
# print(df_near)
# print("\nDataFrame de casas lejanas:")

# FORMA MEJORADA